<a href="https://colab.research.google.com/github/cras-lab/ForMe/blob/main/Consistency_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install -q XlsxWriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 3.2 MB/s eta 0:00:00


In [97]:
from google.colab import files
from google.colab import drive
import pandas as pd
import numpy as np
import math
import shutil
import os

USE_FIXED_FILE = False

#SHEET_NAME = [  'AI 전문가', '일반 전문가', 'CEO', '전체' ]
#idx_label = [ ' AI_L', 'AI_W','AI_GW', '일반L', '일반W', '일반GW', 'CEO_L', 'CEO_W', 'CEO_GW', '전체L', '전체W', '전체GW' ]

SHEET_NAME = [  'AI 전문가', '일반 전문가', 'CEO', '전체-CEO', '전체' ]
idx_label = [ ' AI_L', 'AI_W','AI_GW', '일반L', '일반W', '일반GW', 'CEO_L', 'CEO_W', 'CEO_GW', '전체-CEOL', '전체-CEOW', '전체-CEOGW', '전체L', '전체W', '전체GW' ]

if USE_FIXED_FILE:
  local_file_path = "D:\\Research\\answers.xlsx"
  local_file_path = "C:/nsispromotion_log.txt"
  src_path = "/content/answers.xlsx"
  shutil.copy( local_file_path, src_path )
else:
  uploaded = files.upload()
  filename = list(uploaded.keys())[0]
  src_path = "/content/" + filename

NUM_CLASS = 5
NUM_Q = 4
NUM_COMB = math.comb( NUM_Q, 2)

def GetMatrix( col ):
  matrix = [ [1, col[0], col[1], col[2] ],
             [1/col[0], 1, col[3], col[4]],
             [1/col[1], 1/col[3], 1, col[5]],
             [ 1/col[2], 1/col[4], 1/col[5], 1]
           ]
  return matrix

def FindEigenValue( col ):
  mat = GetMatrix(col)
  eigenvalues, eigenvectors = np.linalg.eig(mat)
  principal_eigenvalue = max(abs(eigenvalues))

  return principal_eigenvalue

def FindEigenValues( col ):
  eigens = np.zeros(NUM_CLASS)

  for i in range( 0, NUM_CLASS):
    col1 = col[i*NUM_COMB:(i+1)*NUM_COMB]
    eigens[i] = FindEigenValue(col1)

  return eigens

# 각 사람별로 가중치를 구함
def GetWeight(col):
  harmonic = np.zeros(NUM_Q*NUM_CLASS)
  weight = np.zeros(NUM_Q*NUM_CLASS)

  for i in range( 0, NUM_CLASS):
    col1 = col[i*NUM_COMB:(i+1)*NUM_COMB]
    mat = GetMatrix(col1)

    for j in range(0, NUM_Q):
      harmonic[i*NUM_Q+j] = (mat[j][0]*mat[j][1]*mat[j][2]*mat[j][3])**(1/NUM_Q)

    total = sum( harmonic[i*NUM_Q:i*NUM_Q+4])

    weight[i*NUM_Q:i*NUM_Q+4 ] = [ x/total for x in harmonic[i*NUM_Q:i*NUM_Q+4]]

  return weight

drive.mount( '/content/drive')
dest_path = '/content/drive/MyDrive/result.xls'


wgt_sorted_total = []
wgt_original_total = []

SERM_weight = np.zeros((len(SHEET_NAME), 4) )

for num in range(0, len(SHEET_NAME)):
  df = pd.read_excel(src_path,sheet_name=SHEET_NAME[num] )
  num_persons = len(df.columns)

  e_values = np.zeros( (num_persons-1, NUM_CLASS) )

  CI = np.zeros( (num_persons-1, NUM_CLASS) )
  Wgt = np.zeros( (num_persons-1, NUM_CLASS*NUM_Q ) )

  for i in range(0, num_persons-1):
      col = df[ df.columns[i+1]].values
      e_values[i] = FindEigenValues( col )  # Primary Eigen value 추출
      CI[i] = [ (x-4)/3/0.9 for x in e_values[i] ]
      Wgt[i] = GetWeight(col)

  # 편의상, Sorting하기전에 SERM의 가중치만 미리 저장해 둔다.
  SERM_weight[num] = Wgt[-1, 0:4 ]

  ev_df = pd.DataFrame( e_values )
  ev_df.columns = [ 'SERM', 'Subject', 'Enviroment', 'Resource', 'Mechanism']
  ev_df.set_index( df.columns[1:],  inplace=True)

  CI_df = pd.DataFrame( CI )
  CI_df.columns = [ 'SERM', 'Subject', 'Enviroment', 'Resource', 'Mechanism']
  CI_df.set_index( df.columns[1:],  inplace=True)

  Wgt_df = pd.DataFrame( Wgt )
  Wgt_df.set_index( df.columns[1:],  inplace=True)
  Wgt_df.columns = [ 'S', 'E', 'R', 'M', 'S1', 'S2', 'S3', 'S4', 'E1', 'E2', 'E3', 'E4', 'R1', 'R2', 'R3', 'R4', 'M1', 'M2', 'M3', 'M4']

  Wgt_sorted = pd.DataFrame()   # 가중치에 따라 정렬
  Wgt_original = pd.DataFrame()   # 정렬하지 않은 원래 데이터

  for idx in range(0, len(Wgt_df.columns), 4):
    group = Wgt_df.iloc[:, idx:idx+4]
    last_row = group.iloc[-1]
    sorted_group = group.sort_values(by=last_row.name, axis=1, ascending=False)

    Wgt_original = pd.concat( [Wgt_original, group], axis=1)
    Wgt_sorted = pd.concat( [Wgt_sorted, sorted_group], axis=1)

  wgt_sorted_total.append( Wgt_sorted.columns.values )
  wgt_sorted_total.append( Wgt_sorted.iloc[-1, :])
  wgt_original_total.append( Wgt_original.columns.values )
  wgt_original_total.append( Wgt_original.iloc[-1, :])

  # 글로벌 가중치를 계산한다. 각 항목에 S,E,R,M의 가중치를 곱한다.
  global_w_total =   Wgt_sorted.values[-1, :]
  global_original = Wgt_original.values[-1, :]

  for k in range(1, 5):
    global_w_total[ NUM_Q*k:NUM_Q*k+4 ] = [ x * SERM_weight[num, k-1] for x in global_w_total[ NUM_Q*k:NUM_Q*k+4 ] ]
    global_original[ NUM_Q*k:NUM_Q*k+4 ] = [ x * SERM_weight[num, k-1] for x in global_original[ NUM_Q*k:NUM_Q*k+4 ] ]

  wgt_original_total.append( global_original )
  wgt_sorted_total.append( global_w_total )
  tmp_path = "/content/temp.xls"

  if num == 0:  # 처음 호출 , 파일 생성
    with pd.ExcelWriter(tmp_path, engine='xlsxwriter') as writer:
    #    ev_df.to_excel(writer, sheet_name='Eigen Values', index=True)
      CI_df.T.to_excel(writer, sheet_name='CI_' + SHEET_NAME[num] , index=True)
      Wgt_sorted.T.to_excel(writer, sheet_name='Weight_' + SHEET_NAME[num], index=True)
  else:   # 파일 이미 존재
    with pd.ExcelWriter( tmp_path, engine='openpyxl', mode='a') as writer:
  #    ev_df.to_excel(writer, sheet_name='Eigen Values', index=True)
      CI_df.T.to_excel(writer, sheet_name='CI_' + SHEET_NAME[num] , index=True)
      Wgt_sorted.T.to_excel(writer, sheet_name='Weight_' + SHEET_NAME[num], index=True  )
wgt_sorted_total_df = pd.DataFrame(wgt_sorted_total )
wgt_original_total_df = pd.DataFrame(wgt_original_total )

wgt_original_total_df.set_index( pd.Index(idx_label), inplace=True )
wgt_sorted_total_df.set_index( pd.Index(idx_label), inplace=True )

with pd.ExcelWriter( tmp_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
  wgt_original_total_df.T.to_excel(writer, sheet_name='Total', index=True, startrow=1, startcol=1, float_format="%.3f")
  wgt_sorted_total_df.T.to_excel(writer, sheet_name='Total', index=True, startcol=len(wgt_original_total_df)+4, startrow=1, float_format="%.3f")

  ws = writer.sheets['Total']
  ws.cell(column=1, row=1, value=' 정렬하지 않은 값')
  ws.cell(column=len(wgt_original_total_df)+6, row=1, value='요인별로 정렬한 값')


# SERM 가중치를 곱한 전체 가중치에 따른 순서로 sorting한 다음 출력한다.
Global_sorted_by_factor = pd.DataFrame()   # 가중치에 따라 정렬

for idx in range(0, len(wgt_sorted_total_df.columns), 4):
  group = wgt_sorted_total_df.iloc[:, idx:idx+4]
  last_row = group.iloc[-1]
  sorted_group = group.sort_values(by=last_row.name, axis=1, ascending=False)
  Global_sorted_by_factor = pd.concat( [Global_sorted_by_factor, sorted_group], axis=1)

# Factor별이 아닌 글로벌 전체 순위를 정렬한다.
Global_sorted = pd.DataFrame()
group = wgt_sorted_total_df.iloc[:, 4:]
sorted_group = group.sort_values(by=last_row.name, axis=1, ascending=False)
Global_sorted = pd.concat( [Global_sorted, wgt_sorted_total_df.iloc[:, :4] ], axis=1)
Global_sorted = pd.concat( [Global_sorted, sorted_group], axis=1)

with pd.ExcelWriter( tmp_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
  Global_sorted_by_factor.T.to_excel(writer, sheet_name='GW_Sorted', index=True, startrow=1, startcol=1, float_format="%.3f")
  Global_sorted.T.to_excel(writer, sheet_name='GW_Sorted', index=True, startcol=len(Global_sorted_by_factor)+4, startrow=1, float_format="%.3f")

  ws = writer.sheets['GW_Sorted']
  ws.cell(row=1, column=2, value='S,E,R,M 각 요인별로 Local 가중치 정렬')
  ws.cell(column=len(Global_sorted_by_factor)+6, row=1, value='전체 요인을 Global 가중치로 정렬')


shutil.copy( tmp_path , dest_path )


Saving answers.xlsx to answers (85).xlsx
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/result.xls'